# OpenEye (OE) docking example

This notebook shows how to process data from [KLIFS](https://klifs.vu-compmedchem.nl/), perform docking into a kinase with different docking methods available in the [OpenEye toolkit](https://docs.eyesopen.com/toolkits/python/index.html) as well as how to assess and visualize docking results.

The main aim of the following sections is to get to know two docking methods (Hybrid, ChemGauss) from OEDocking. The [Hybrid](https://docs.eyesopen.com/toolkits/python/dockingtk/docking.html#hybrid-method) method uses a shape overlay to the co-crystalized ligand to bias the docking algorithm. However, it is not usable if a co-crystalized ligand is not available. In this situation, the more 'traditional' [ChemGauss](https://docs.eyesopen.com/toolkits/python/dockingtk/scoring.html#chemgauss3) docking method can be used to dock a ligand.

Re-docking will be performed to assess the performance of the two docking methods in placing the co-crystalized ligand in the corresponding protein structure. Cross-docking will be performed to assess the performance of the Hybrid method in placing a ligand into another structure of the same kinase and conformation. Here it would be interesting to evaluate, if the Hybrid method fails if the docked ligand is too dissimilar to the co-crystalized ligand used during shape overlay.

Docking results will be visualized using NGLView and assessed by calculating the RMSD between docking pose and crystal structure pose.

## Content

1. Process data from KLIFS.  
2. Re-Docking.  
   2.1. Prepare protein and ligand.  
   2.2. Hybrid docking method.  
   2.3. ChemGauss docking method.  
3. Cross-Docking.

## 1. Process data from KLIFS.

Let's retrieve all relevant data from KLIFS and store it in a single dataframe for subsequent processing.

In [1]:
import klifs_utils
kinase_ids = klifs_utils.remote.kinases.kinase_names().kinase_ID.to_list()
kinase_df = klifs_utils.remote.structures.structures_from_kinase_ids(kinase_ids)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))
kinase_df.tail()

Number of PDB entries: 5135
Number of KLIFS entries: 11070


,structure_ID,kinase,species,kinase_ID,pdb,alt,chain,rmsd1,rmsd2,pocket,...,bp_I_A,bp_I_B,bp_II_in,bp_II_A_in,bp_II_B_in,bp_II_out,bp_II_B,bp_III,bp_IV,bp_V
11065,9096,PI4K2B,Human,1091,4wtv,B,B,1.554,3.041,ERIS___SGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11066,9095,PI4K2B,Human,1091,4wtv,A,A,1.554,3.088,ERISQGSSGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11067,9097,PI4K2B,Human,1091,4wtv,A,B,1.554,3.041,ERIS___SGSYFVGVFKPKGYLSEAGAYLVDNSIVPKTKVVGSFQL...,...,False,False,False,False,False,False,False,False,False,False
11068,9069,PI4KA,Human,1096,6bq1,,E,1.699,2.670,_PMQSAAKAPYLAAIFKVGDCRQDMLALQIIDLFVFPYRVVCGVIE...,...,True,True,False,False,False,False,False,False,False,False
11069,9070,PI4KA,Human,1096,6bq1,,A,1.704,2.676,_PMQSAAKAPYLAAIFKVGDCRQDMLALQIIDLFVFPYRVVCGVIE...,...,False,True,False,False,False,False,False,False,False,False


Interestingly, we find several entries for the same PDB ID. Why is that?

In [2]:
kinase_df.columns

Index(['structure_ID', 'kinase', 'species', 'kinase_ID', 'pdb', 'alt', 'chain',
       'rmsd1', 'rmsd2', 'pocket', 'resolution', 'quality_score',
       'missing_residues', 'missing_atoms', 'ligand', 'allosteric_ligand',
       'DFG', 'aC_helix', 'Grich_distance', 'Grich_angle', 'Grich_rotation',
       'front', 'gate', 'back', 'fp_I', 'fp_II', 'bp_I_A', 'bp_I_B',
       'bp_II_in', 'bp_II_A_in', 'bp_II_B_in', 'bp_II_out', 'bp_II_B',
       'bp_III', 'bp_IV', 'bp_V'],
      dtype='object')

**alt** - Structures can contain alternate locations of atoms.  
**chain** - Structures can contain several chains with different geometry.  
**ligand**, **allosteric_ligand** - Structures can contain multiple ligands either in the same binding site or in different chains. In case of 6q0t there are two ligands interacting with each other, docking these ligands seperately might be challenging. In case of 1h00 the racemic mixture was crystalized revealing both enantiomers in the binding pocket, which would be suitable for docking.

To make things easier, let's pick a single entry per pdb code. Such entry should contain a single orthosteric **ligand** and should have the highest **quality_score** of all available **alt/chain combinations**.

In [3]:
# filter for orthosteric ligand
kinase_df = kinase_df[kinase_df['ligand'] != 0]
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4504
Number of KLIFS entries: 9389


In [4]:
# filter for entries with a single orthosteric ligand
kinase_df = kinase_df.groupby('pdb').filter(lambda x: len(set(x['ligand'])) == 1)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4479
Number of KLIFS entries: 9280


In [5]:
# sort by alt, chain and quality score to pick representative structure in next step
kinase_df = kinase_df.sort_values(by=['alt','chain', 'quality_score'],ascending=[True, True, False])
# keep entry with highest quality score (alt 'A' preferred over alt 'B', chain 'A' preferred over 'B')
kinase_df = kinase_df.groupby('pdb').head(1)
print('Number of PDB entries:', len(set(kinase_df['pdb'])))
print('Number of KLIFS entries:', len(kinase_df))

Number of PDB entries: 4479
Number of KLIFS entries: 4479


This data set contains bullet proof cases, that an ideal docking algorithm should be able to re-dock. Let's save it for later use.

In [6]:
kinase_df.to_csv('data/re_docking_data.csv')

Another challenge for a docking program would be cross-docking, in which a co-crystalized ligand is docked into another protein structure corresponding to the same kinase. Since kinases are highly dynamic, cross-docking should only be performed within the same kinase conformations. In KLIFS unqiue kinases can be identified using **kinase_ID**. The conformation can be determined with the **DFG** and **aC_helix** fields.

In [7]:
# find structures, for which multiple entries per kinase and conformation are available
multiple_pdbs_df = kinase_df.groupby(['kinase_ID', 'DFG', 'aC_helix']).filter(lambda x: len(x[['kinase_ID', 'DFG', 'aC_helix']]) > 1)

In [8]:
# Check if command works
kinase_dict = {}
for index, row in multiple_pdbs_df.iterrows():
    identity = str(row['kinase_ID']) + row['DFG'] + row['aC_helix']
    if identity in kinase_dict.keys():
        kinase_dict[identity].append(index)
    else:
        kinase_dict[identity] = [index]
print(f'Minimum number of entries for each kinase and conformation: {min([len(x) for x in kinase_dict.values()])}')
print('Number of PDB entries:', len(set(multiple_pdbs_df['pdb'])))
print('Number of KLIFS entries:', len(multiple_pdbs_df))

Minimum number of entries for each kinase and conformation: 2
Number of PDB entries: 4281
Number of KLIFS entries: 4281


Let's save this data set for later use.

In [9]:
multiple_pdbs_df.to_csv('data/cross_docking_data.csv')

## 2. Re-Docking.

### 2.1. Prepare protein and ligand.

In this step a structure will be retrieved from PDB, and protein and ligand prepared for docking.

In [10]:
# pick structure
pdb_id = '1jvp'
chain_id, altloc, ligand_id  = kinase_df[kinase_df['pdb'] == pdb_id][['chain', 'alt', 'ligand']].values[0]
print(f'pdb: {pdb_id}\nchain: {chain_id}\naltloc: {altloc}\nligand: {ligand_id}')

pdb: 1jvp
chain: P
altloc: 1
ligand: LIG


In [11]:
# retrieve structure
from dockin.oe_docking import get_structure_from_pdb
mol = get_structure_from_pdb(pdb_id)
mol.NumAtoms()

2709

In [12]:
# select chain, alternate location and ligand
from dockin.oe_docking import select_chain, select_altloc, select_ligand
mol = select_chain(mol, chain_id)
mol = select_altloc(mol, altloc)
mol = select_ligand(mol, ligand_id)
mol.NumAtoms()

2547

In [13]:
# prepare protein and ligand and save it for later visualization
from dockin.oe_docking import prepare_complex
protein, ligand = prepare_complex(mol, f'data/{pdb_id}_protein.pdb', f'data/{pdb_id}_{ligand_id}.sdf')
print(f'Protein: {protein.NumAtoms()}\nLigand: {ligand.NumAtoms()}')

Re-optimizing hydrogen positions...
Identifying design units...
Protein: 4631
Ligand: 29


### 2.2. Hybrid docking method.

Now we have everything to perform a Re-Docking using the Hybrid method.

In [14]:
# create hybrid 
from dockin.oe_docking import create_hybrid_receptor
hybrid_receptor = create_hybrid_receptor(protein, ligand, f'data/{pdb_id}_hybrid_receptor.oeb')

In [15]:
# run docking
from dockin.oe_docking import hybrid_docking
# the just extracted ligand will be passed as list, since the hybrid_docking function expects a list of molecules
docking_poses = hybrid_docking(hybrid_receptor, [ligand], docking_poses_save_path=f'data/{pdb_id}_hybrid_re-docking.sdf')

Let's visualize the docking pose together with the co-crystalized ligand.

In [16]:
#  visualize with nglview
import nglview as nv
view = nv.NGLWidget()
view.add_component(f'data/{pdb_id}_protein.pdb')
view.add_component(f'data/{pdb_id}_{ligand_id}.sdf')
view.add_component(f'data/{pdb_id}_hybrid_re-docking.sdf', representation='licorice')
view

_ColormakerRegistry()

NGLWidget()

What is the RMSD between docking pose and co-crystalied ligand?

In [17]:
# calculate rmsd
from dockin.assess import rmsds_from_sdfs
rmsd = rmsds_from_sdfs(f'data/{pdb_id}_{ligand_id}.sdf', f'data/{pdb_id}_hybrid_re-docking.sdf')[0]
print(f'RMSD: {rmsd}')

RMSD: 0.7331801061933088


### 2.2. ChemGauss docking method.

The OEDocking toolkit provides the possibility to generate [receptors](https://docs.eyesopen.com/toolkits/python/dockingtk/receptor.html#creating-a-receptor) for docking via protein and ligand (hybrid), protein and hint coordinate or via protein and box dimensions of the active site. In this example we will use the geometric center of the co-crystalized ligand to create a recepter with a hint coordinate.

In [18]:
# get geometric center of ligand
from dockin.oe_docking import get_mol_centroid
hintx, hinty, hintz = get_mol_centroid(ligand)

In [19]:
# create hint receptor
from dockin.oe_docking import create_hint_receptor
hint_receptor = create_hint_receptor(protein, hintx, hinty, hintz, f'data/{pdb_id}_hint_receptor.oeb')

In [20]:
# run docking
from dockin.oe_docking import chemgauss_docking
docking_poses = chemgauss_docking(hint_receptor, [ligand], docking_poses_save_path=f'data/{pdb_id}_hint_re-docking.sdf')

In [21]:
#  visualize with nglview
import nglview as nv
view = nv.NGLWidget()
view.add_component(f'data/{pdb_id}_protein.pdb')
view.add_component(f'data/{pdb_id}_{ligand_id}.sdf')
view.add_component(f'data/{pdb_id}_hint_re-docking.sdf')
view

NGLWidget()

In [22]:
# calculate rmsd
rmsd = rmsds_from_sdfs(f'data/{pdb_id}_{ligand_id}.sdf', f'data/{pdb_id}_hint_re-docking.sdf')[0]
print(f'RMSD: {rmsd}')

RMSD: 0.7331598508659334


In this case hybrid and chemgauss docking produced very similar results.

## 3. Cross-Docking.

In cross-docking a ligand will be docked into a different structure matching the same kinase and conformation. Again, we want to visualize the results with NGLView and calculate an RMSD between docking pose and co-crystalized ligand.

First, lets find a suitable structure for cross-docking. We used the structure with the pdb code `1jvp` in the re-docking experiment, so lets first find another structure from the same kinase type in the same kinase conformation.

In [23]:
# get kinase information from ṕdb '1jvp'
kinase_id, ac_helix, dfg = kinase_df[kinase_df['pdb'] == pdb_id][['kinase_ID', 'aC_helix', 'DFG']].values[0]
# retrieve structure from the same type and with the same conformation and pick the first one
kinase2 = kinase_df[(kinase_df['kinase_ID'] == kinase_id) & (kinase_df['aC_helix'] == ac_helix) & 
          (kinase_df['DFG'] == dfg)].iloc[0]
# get kinase information for further processing
pdb_id2, chain_id2, altloc2, ligand_id2 = kinase2[['pdb', 'chain', 'alt', 'ligand']].values
print(f'pdb: {pdb_id2}\nchain: {chain_id2}\naltloc: {altloc2}\nligand: {ligand_id2}')

pdb: 1oit
chain: A
altloc: 
ligand: HDT


Just like before, let's retrieve the corresponding structure from pdb and select the chain and ligand for further processing.

In [24]:
# retrieve structure from pdb
mol = get_structure_from_pdb(pdb_id2)
# select chain and ligand, altloc not given
mol = select_chain(mol, chain_id2)
mol = select_ligand(mol, ligand_id2)

Next, align the novel structure to the protein of `1jvp` from above re-docking experiments.

In [25]:
from dockin.oe_docking import superpose_proteins
rmsd, superposed_protein = superpose_proteins(protein, mol, 
    superposed_protein_save_path=f'data/{pdb_id}_{pdb_id2}_superposed.pdb')
rmsd

0.4103922767769461

Prepare protein and ligand from superposed_protein.

In [26]:
protein2, ligand2 = prepare_complex(superposed_protein, f'data/{pdb_id2}_protein.pdb', f'data/{pdb_id2}_{ligand_id2}.sdf')

Re-optimizing hydrogen positions...
Identifying design units...


Run docking experiments.

In [27]:
docking_poses = hybrid_docking(hybrid_receptor, [ligand2], 
                               docking_poses_save_path=f'data/{pdb_id}_{ligand_id2}_hybrid_cross-docking.sdf')
docking_poses = chemgauss_docking(hint_receptor, [ligand2], 
                                  docking_poses_save_path=f'data/{pdb_id}_{ligand_id2}_chemgauss_cross-docking.sdf')

Calculate RMSDs between docked compound and co-crystalized binding pose.

In [28]:
rmsd_hybrid = rmsds_from_sdfs(f'data/{pdb_id2}_{ligand_id2}.sdf', f'data/{pdb_id}_{ligand_id2}_hybrid_cross-docking.sdf')[0]
rmsd_chemgauss = rmsds_from_sdfs(f'data/{pdb_id2}_{ligand_id2}.sdf', f'data/{pdb_id}_{ligand_id2}_chemgauss_cross-docking.sdf')[0]
print(f'RMSD:\nHybrid - {rmsd_hybrid}\nChemGauss - {rmsd_chemgauss}')

RMSD:
Hybrid - 0.7408726809157818
ChemGauss - 0.9775361042732151


Visualize dockin results with NGLView.

In [29]:
#  visualize with nglview
import nglview as nv
view = nv.NGLWidget()
view.add_component(f'data/{pdb_id}_{pdb_id2}_superposed.pdb')
view.add_component(f'data/{pdb_id}_{ligand_id2}_hybrid_cross-docking.sdf')
view.add_component(f'data/{pdb_id}_{ligand_id2}_chemgauss_cross-docking.sdf')
view

NGLWidget()

In [30]:
view.clear_representations(component=0)
view.clear_representations(component=1)
view.clear_representations(component=2)
view.add_representation('cartoon', selection='protein', component=0)
view.add_representation('licorice', selection=ligand_id2, component=0)
view.add_representation('licorice', component=1, selection='not hydrogen', color='red')
view.add_representation('licorice', component=2, selection='not hydrogen', color='green')
print('Hybrid - red\nChemGauss - green')

Hybrid - red
ChemGauss - green


Both, Hybrid an ChemGauss docking, were able to reproduce the observed binding mode of the co-crystalied ligand during cross-docking.